# Random Walk Metrics


## Modules

In [4]:
import numpy as np
import pandas as pd
import math

from scipy.spatial import distance

from scipy.stats  import wrapcauchy
from scipy.stats import levy_stable

import plotly.graph_objects as go


## Class

In [5]:
################# http://www.pygame.org/wiki/2DVectorClass ##################
class Vec2d(object):
    """2d vector class, supports vector and scalar operators,
    and also provides a bunch of high level functions
    """
    __slots__ = ['x', 'y']

    def __init__(self, x_or_pair, y = None):
        if y == None:            
            self.x = x_or_pair[0]
            self.y = x_or_pair[1]
        else:
            self.x = x_or_pair
            self.y = y
            
    # Addition
    def __add__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x + other.x, self.y + other.y)
        elif hasattr(other, "__getitem__"):
            return Vec2d(self.x + other[0], self.y + other[1])
        else:
            return Vec2d(self.x + other, self.y + other)

    # Subtraction
    def __sub__(self, other):
        if isinstance(other, Vec2d):
            return Vec2d(self.x - other.x, self.y - other.y)
        elif (hasattr(other, "__getitem__")):
            return Vec2d(self.x - other[0], self.y - other[1])
        else:
            return Vec2d(self.x - other, self.y - other)
    
    # Vector length
    def get_length(self):
        return math.sqrt(self.x**2 + self.y**2)
    
    # rotate vector
    def rotated(self, angle):        
        cos = math.cos(angle)
        sin = math.sin(angle)
        x = self.x*cos - self.y*sin
        y = self.x*sin + self.y*cos
        return Vec2d(x, y)

## Activity 1 - Path length - (BM1 vs BM2 vs CRW)
    - Write a function that returns a Brownian Motion (BM) trajectory in pandas df.
    - Write a function that returns a Correlated Random Walk (CRW) trajectory in pandas df.
    - Write a function that returns the path length for a given trajectory.
    - Compare at least the path length of three trajectories as shown in the figure below.
    - Display the results using plotly.

In [6]:
# Function to get Brownian Motion trajectory, using random 90 degree rotation angle directions
def bm_traj(speed=3, n_steps=1000, s_pos=[0,0]):
    trajectory_bw = pd.DataFrame(columns=['x_pos','y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1]}])

    trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

    velocity = Vec2d(speed, 0) # Starting position for velocity vector

    for i in range(n_steps):
        turn_angle = np.random.choice([0, np.pi/2, np.pi, 3*np.pi/2])
        
        velocity = velocity.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{'x_pos':trajectory_bw.x_pos[i] + velocity.x, 'y_pos':trajectory_bw.y_pos[i] + velocity.y}])
        trajectory_bw = pd.concat([trajectory_bw, temp_df], ignore_index=True)

    return trajectory_bw
    

In [7]:
# Function to get correlated random walk trajectory using Wrapped Cauchy Distribution for turning angles on each step/iteration
def crw_traj(speed=5, coefficient=0.4, n_steps=1000, s_pos=[0,0]):
    
    r = wrapcauchy.rvs(loc=0, c=coefficient, size=n_steps)

    velocity = Vec2d(speed, 0) # Velocity vector in starting position

    trajectory_crw = pd.DataFrame(columns=['x_pos', 'y_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0],'y_pos':s_pos[1]}])

    trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True) 

    for i in range(n_steps):
        turn_angle = r[i]
        
        velocity = velocity.rotated(turn_angle)
        
        temp_df = pd.DataFrame([{'x_pos':trajectory_crw.x_pos[i] + velocity.x, 'y_pos':trajectory_crw.y_pos[i] + velocity.y}])
        trajectory_crw = pd.concat([trajectory_crw,temp_df], ignore_index=True)
        
    return trajectory_crw
    

In [8]:
# Function to get the path length of a given dataframe trajectory
def pl_traj(df, s_pos=[0]):
    pl = pd.DataFrame(columns=['y_pos'])
    temp_df = pd.DataFrame([{'y_pos':s_pos[0]}])

    pl = pd.concat([pl, temp_df], ignore_index=True)

    for i in range(len(df)-1): # minus one because len counts the dataframe header
        temp_df = pd.DataFrame([{'y_pos':pl.y_pos[i] + distance.euclidean(df.iloc[i], df.iloc[i+1])}])
        pl = pd.concat([pl, temp_df], ignore_index=True)
        
    return pl


In [9]:
# Obtaining the correspondent trajectories
bm_tr_3 = bm_traj(3)
bm_tr_6 = bm_traj(6)
crw_tr_6 = crw_traj(6, coefficient=0.9)

# Obtaining path length dataframe for each trajectory
bm_pl_3 = pl_traj(bm_tr_3)
bm_pl_6 = pl_traj(bm_tr_6)
crw_pl_6 = pl_traj(crw_tr_6)

In [10]:
# Plotting of three path length trajectories for both Brownian Motion and Correlated Random Walks
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= bm_pl_3.index,
    y= bm_pl_3.y_pos,
    name='BM - 3',
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= bm_pl_6.index,
    y= bm_pl_6.y_pos,
    name='BM - 6',
    marker = dict(size=2),
    line = dict(width=6),
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= crw_pl_6.index,
    y= crw_pl_6.y_pos,
    name='CRW 0.9 - 6',
    mode='lines',
    showlegend=True
))

fig.update_layout(
    title='Path Lengths',
    height=800,
    xaxis_title="times",
    yaxis_title="distnace",
)

fig.show()
    

## Mean Squared Displacement - (BM vs CRW)
    - Write a function that returns the mean squared displacement for a given trajectory.
    - Compare the mean squared displacement curves of at least two trajectories of
    different kinds, as shown in the figure below.
    - Display the results using plotly.

In [11]:

def msd(df, s_pos=0):
    n = 1 # MSD Formula n initial value
    msd_df = pd.DataFrame(columns=['y_pos'])
    temp_df = pd.DataFrame([{'y_pos':s_pos}])
    msd_df = pd.concat([msd_df, temp_df], ignore_index=True)
    
    for n in range(1,len(df)-1):
        
        eucladean_distances = np.array([distance.euclidean(df.iloc[i-n],df.iloc[i]) for i in range(n, df.shape[0],1)])
        dist_squared = eucladean_distances**2
        msd = np.mean(dist_squared)
        
        
        temp_df = pd.DataFrame([{'y_pos':msd}])
        msd_df = pd.concat([msd_df, temp_df], ignore_index=True)
        
    return msd_df

bw_msd_df = msd(bm_tr_3)
crw_msd_df = msd(crw_tr_6)
fig = go.Figure()

fig.add_trace(go.Scatter(
    x= bw_msd_df.index,
    y= bw_msd_df.y_pos,
    name='BM - 3 - MSD',
    mode='lines',
    showlegend=True
))

fig.add_trace(go.Scatter(
    x= crw_msd_df.index,
    y= crw_msd_df.y_pos,
    name='CRW - 6 - MSD',
    mode='lines',
    showlegend=True
))


fig.update_layout(
    title='Mean Squared Distances',
    height=800,
    xaxis_title="times",
    yaxis_title="msd per position",
)

fig.show()

## Activity 3 Turning-angle Distribution - (source dist. vs observed dist.)
    - Consider two CRW trajectories with different Cauchy coefficients.
    - Write a function that returns the turning angles for a given trajectory.
    - Compare the observed distribution (histogram) to the source distribution (curve) for
    both trajectories, as shown in the figure below.
    - Display the results using plotly. 

In [12]:
# a = bm_tr_6.iloc[21]
# b = bm_tr_6.iloc[22]
# c = bm_tr_6.iloc[23]

# turning_angle(a,b,c)


# ab = b-a
# bc = c-b

# np.linalg.norm(ab)
# np.linalg.norm(bc)

# dot_p = ...
# cross_p = ...

# orient = np.sign(cross_p)
# if orient == 0:
#     orient = 1

# np.finfo(float).eps

# theta*orient

In [14]:

# def turning_angle(a_pos, b_pos, c_pos):
#     ab = b_pos-a_pos
#     bc = c_pos-b_pos

#     np.linalg.norm(ab)
#     np.linalg.norm(bc)

# def turn_angle_traj(crw_df):
#     for i in range (len(crw_df) - 2):
#         a = crw_df.iloc[i]
#         b = crw_df.iloc[i+1]
#         c = crw_df.iloc[i+2]
        
#         turning_angle = turning_angle(a,b,c)
        
# size = 200
# x = np.linspace(-3,3,size)

# fig_crw_pdf = go.Figure()

# fig_crw_pdf.add_trace(go.Scatter(
#     x = x,
#     y = crw_pdf,
#     mode = 'lines',
#     name='CRW - 0.6 PDF',
#     showlegend=True
# ))

# fig_crw_pdf.add_trace(go.Scatter(
#     x = x,
#     y = crw_tr_6.y_pos,
#     mode = 'lines',
#     name='CRW - 0.6 Trejectory',
#     showlegend=True
# ))

# fig_crw_pdf.update_layout(
#     title='Turning Angle Distribution',
#     height=800,
#     xaxis_title="times",
#     yaxis_title="Probability Density Fucntion",
# )

# fig_crw_pdf.show()




In [54]:
# Consider two CRW trajectories with different Cauchy coefficients.
crw_06 = crw_traj(speed=6, coefficient=0.6)
crw_09 = crw_traj(speed=6, coefficient=0.9)
# Write a function that returns the turning angles for a given trajectory.

def turning_angles(trajectory):
    turning_angles = []
    for i in range(1, len(trajectory)-1):
        # Calculate the turning angle
        a = trajectory.iloc[i-1]
        b = trajectory.iloc[i]
        c = trajectory.iloc[i+1]
        ab = b - a
        bc = c - b
        cross_product = np.cross(ab, bc)
        orient = np.sign(cross_product)
        if orient == 0:
            orient = 1
            
        denominateur = np.linalg.norm(ab) * np.linalg.norm(bc) + np.finfo(float).eps #to avoid zeros
        
        angle = np.arcsin(np.linalg.norm(cross_product) / denominateur)
        
        new_angle = angle * orient
        turning_angles.append(round(new_angle, 2))
        
    return pd.DataFrame(turning_angles, columns=['turning_angle'])
# Compare the observed distribution (histogram) to the source distribution (curve) for both trajectories.

# Calculate turning angles for both trajectories

turning_angles_1 = turning_angles(crw_06)
turning_angles_2 = turning_angles(crw_09)



x_values_angles_1 = np.linspace(min(turning_angles_1.turning_angle), max(turning_angles_1.turning_angle), 1000)
y_values_angles_1 = np.array([wrapcauchy.pdf(abs(i), 0.6) for i in x_values_angles_1])

scaling_factor = 100
y_values_angles_1 *= scaling_factor

x_values_angles_2 = np.linspace(min(turning_angles_2.turning_angle), max(turning_angles_2.turning_angle), 1000)
y_values_angles_2 = np.array([wrapcauchy.pdf(abs(i), 0.9) for i in x_values_angles_2])

scaling_factor = 100
y_values_angles_2 *= scaling_factor

# Plot histograms using Plotly
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(
    y=turning_angles_1.index,
    x=turning_angles_1.turning_angle,
    name='CRW 0.6 observable',
    marker_color='blue'
))
fig_hist.add_trace(go.Histogram(
    y=turning_angles_2.index,
    x=turning_angles_2.turning_angle,
    name='CRW 0.9 observable',
    marker_color='red'
))
fig_hist.add_trace(go.Scatter(
    x = x_values_angles_1,
    y = y_values_angles_1,
    mode = 'lines',
    name='CRW - 0.6 PDF',
    showlegend=True
))

fig_hist.add_trace(go.Scatter(
    x = x_values_angles_2,
    y = y_values_angles_2,
    mode = 'lines',
    name='CRW - 0.9 PDF',
    showlegend=True
))
fig_hist.update_layout(title='Turning Angles Histogram')

fig_hist.show()

     turning_angle
450           0.82
451           0.36
452          -0.07
453          -0.28
454          -0.68
455          -0.70
456           0.19
457          -0.34
458          -0.97
459           0.39
460          -0.58
461           0.20
462           0.25
463           0.18
464          -1.34
465          -1.22
466          -0.80
467          -0.31
468          -0.30
469          -0.32
470           0.43
471           1.31
472          -0.26
473           1.12
474           1.56
475          -0.13
476          -0.24
477           0.37
478          -0.23
479           0.26
480          -0.22
481           0.20
482          -0.87
483          -0.06
484          -0.57
485          -0.05
486           0.64
487           1.38
488          -0.39
489          -1.20
490          -0.02
491           0.67
492          -0.05
493          -0.43
494          -0.52
495          -0.56
496           0.48
497          -0.37
498           0.33
499          -0.22


## Activity 4 Step-length Distribution - (source dist. vs observed dist.)
    - Write a function that returns a Lévy Walk (LW) trajectory in pandas df.
    - Consider two LW trajectories with different alpha coefficients.
    - Write a function that restaurants the step lengths for a given trajectory.
    - Compare the observed distribution (histogram) to the source distribution (curve) for
    both trajectories, as shown in the figure below.
    - Display the results using plotly.

In [110]:
# We initalize the variables
speed = 10
s_pos = [0,0,0]
n_steps = 1000
time_per_step = 0.0004

alpha = 1.5
beta = 0
m = 0

def levy_traj(alpha = 1.5, beta=0, m = 0, speed=10,s_pos = [0,0,0], n_steps=1000, time_per_step=0.0004):
    # Initialize the dataFrame to store x, y values to trace trajectory, then set initial position for Data Frame
    trajectory = pd.DataFrame(columns=['x_pos', 'y_pos', 'z_pos'])
    temp_df = pd.DataFrame([{'x_pos':s_pos[0], 'y_pos':s_pos[1], 'z_pos':s_pos[2]}]) 

    trajectory = pd.concat([trajectory,temp_df], ignore_index=True)

    # Initialize the velocity vector with initial position
    velocity = Vec2d(speed, 0)

    # Random values using Lévy algorithm
    r = levy_stable.rvs(alpha, beta, loc=m, size=n_steps) #angle rotation values

    # trajectory x and y values are defined by lévy distribution random values.
    # On each iteration the difference between each step is calculated multiplying the value returned by the velocity 
    # vector with the current value of the random lavue from lévy distribution.
    for i in range(n_steps):
        step_length = levy_stable.rvs(alpha,beta)
        
        # Rotation angles for the vector are also random values from lévy distribution
        velocity = velocity.rotated(r[i])
        
        length_traveled = velocity.get_length() * step_length * time_per_step # times values (z)
        
        # new position values are stored in the trajectory dataframe
        temp_df = pd.DataFrame([{
            'x_pos':trajectory.x_pos[i] + velocity.x * step_length, 
            'y_pos':trajectory.y_pos[i] + velocity.y * step_length, 
            'z_pos':trajectory.z_pos[i] + length_traveled
            }]
        )
        trajectory = pd.concat([trajectory, temp_df], ignore_index=True)
        
    return trajectory, r

def step_lengths(lev_traj, rot_ang):
    step_lengths = []
    step_length = 1
    for i in range(len(lev_traj)-1):
        if (rot_ang[i] < 0.01 and rot_ang[i] > -0.1 ):
            step_length += step_length + 1
        else:
            step_lengths.append(step_length)
            step_length = 1

    return pd.DataFrame(step_lengths, columns=['step_length'])
            
levy_traj_1, rot_angles = levy_traj(alpha=1, beta=1)

levy_1_sls = step_lengths(levy_traj_1, rot_angles)

times = np.linspace(0,50, len(levy_1_sls))
levy_pdf_1 = levy_stable.pdf(times, alpha=1, beta=1,loc=5)

# Plot histograms using Plotly
fig_hist = go.Figure()
fig_hist.add_trace(go.Histogram(
    y=times,
    x=levy_1_sls.step_length,
    name='Levy - Alpha 1.0 Beta 1.0 observable',
    marker_color='blue'
))
fig_hist.add_trace(go.Scatter(
    x = times,
    y = levy_pdf_1,
    mode = 'lines',
    name='CRW - 0.6 PDF',
    showlegend=True
))

fig_hist.show()

levy_1_sls

,step_length
0,1
1,1
2,1
3,1
4,1
...,...
886,1
887,1
888,1
889,1
